# A guide to fine-tuning Code Llama

**In this guide I show you how to fine-tune Code Llama to become a beast of an SQL developer. For coding tasks, you can generally get much better performance out of Code Llama than Llama 2, especially when you specialise the model on a particular task:**

- I use the [b-mc2/sql-create-context](https://huggingface.co/datasets/b-mc2/sql-create-context) which is a bunch of text queries and their corresponding SQL queries
- A Lora approach, quantizing the base model to int 8, freezing its weights and only training an adapter
- Much of the code is borrowed from [alpaca-lora](https://github.com/tloen/alpaca-lora), but I refactored it quite a bit for this


### 2. Pip installs


In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes accelerate==0.20.3  # we need latest transformers for this
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08
!pip install datasets==2.10.1
import locale # colab workaround
locale.getpreferredencoding = lambda: "UTF-8" # colab workaround
!pip install wandb
!pip install scipy

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /var/tmp/pip-req-build-en62x6_h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /var/tmp/pip-req-build-en62x6_h
  Resolved https://github.com/huggingface/transformers.git to commit 5936c8c57ccb2bda3b3f28856a7ef992c5c9f451
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/1e/2c/af22cd797fc368a9f098ed03015730e6568b884fe67f9940793d944a4b7b/bitsandbytes-0.41.1-py3-none-any.whl.metadata
  Using cached bitsandbytes-0.41.1-py3-none-any.whl.metadata (9.8 kB)
  Obtaining dependency information for accelerate==0.20.3 from https://files.pythonhosted.org/packages/10/d3/5382aa337d3e67214003a17b06bfc07cf0334356b4e8aaf3b12b0d38c83f/accelerate-0.20.3-py3-none-any.whl.metadata
  Using cached

I used an A100 GPU machine with Python 3.10 and cuda 11.8 to run this notebook. It took about an hour to run.

### Loading libraries


In [1]:
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq


(If you have import errors, try restarting your Jupyter kernel)


### Load dataset


In [2]:
from datasets import load_dataset
train_dataset = load_dataset('json', data_files='/home/sam/finetune-llm-for-rag/datasets/sql/MiniLM-L6/samlhuillier-sql-create-context-spider-intersect-train-with-2-examples.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='/home/sam/finetune-llm-for-rag/datasets/sql/MiniLM-L6/samlhuillier-sql-create-context-spider-intersect-validation-with-2-examples.jsonl', split='train')
print(train_dataset)
print(eval_dataset)

Found cached dataset json (/home/sam/.cache/huggingface/datasets/json/default-8e74ab5de84b7560/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
Found cached dataset json (/home/sam/.cache/huggingface/datasets/json/default-d78df93a57db14be/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


Dataset({
    features: ['db_id', 'context', 'question', 'answer', 'full_prompt', 'inference_prompt'],
    num_rows: 3961
})
Dataset({
    features: ['db_id', 'context', 'question', 'answer', 'full_prompt', 'inference_prompt'],
    num_rows: 568
})


The above pulls the dataset from the Huggingface Hub and splits 10% of it into an evaluation set to check how well the model is doing through training. If you want to load your own dataset do this:

```
train_dataset = load_dataset('json', data_files='train_set.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='validation_set.jsonl', split='train')
```

And if you want to view any samples in the dataset just do something like:``` ```


In [3]:
print(train_dataset[3])

{'db_id': 'department_management', 'context': 'CREATE TABLE department (budget_in_billions INTEGER)', 'question': 'What are the maximum and minimum budget of the departments?', 'answer': 'SELECT max(budget_in_billions) ,  min(budget_in_billions) FROM department', 'full_prompt': 'You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.\n\nGiven the following examples:\nExample 1:\n### Input:\nList the creation year, name and budget of each department.\n\n### Context:\nCREATE TABLE department (creation VARCHAR, name VARCHAR, budget_in_billions VARCHAR)\n\n### Response:\nSELECT creation ,  name ,  budget_in_billions FROM department\n\nExample 2:\n### Input:\nIn which year were most departments established?\n\n### Context:\nCREATE TABLE department (creation VARCHAR)\n\n### Response:\nSELECT creation FROM department GROUP BY creation ORDE

Each entry is made up of a text 'question', the sql table 'context' and the 'answer'.

### Load model
I load code llama from huggingface in int8. Standard for Lora:

In [4]:
base_model = "microsoft/phi-1_5"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


torch_dtype=torch.float16 means computations are performed using a float16 representation, even though the values themselves are 8 bit ints.

If you get error "ValueError: Tokenizer class CodeLlamaTokenizer does not exist or is not currently imported." Make sure you have transformers version is 4.33.0.dev0 and accelerate is >=0.20.3.


### 3. Check base model
A very good common practice is to check whether a model can already do the task at hand. Fine-tuning is something you want to try to avoid at all cost:


In [5]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""
# {'question': 'Name the comptroller for office of prohibition', 'context': 'CREATE TABLE table_22607062_1 (comptroller VARCHAR, ticket___office VARCHAR)', 'answer': 'SELECT comptroller FROM table_22607062_1 WHERE ticket___office = "Prohibition"'}
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:

SELECT * FROM table_name_12 WHERE frequency_mhz = 91.5 AND city_of_license = nebraska

### Output:

SELECT * FROM table_name_12 WHERE frequency_mhz = 91.5 AND city_of_license = nebraska

### Output:

SELECT * FROM table_name_12 WHERE frequency_mhz = 91.5 AND city_of_license = nebraska

### Output:




I get the output:
```
SELECT * FROM table_name_12 WHERE class > 91.5 AND city_of_license = 'hyannis, nebraska'
```
which is clearly wrong if the input is asking for just class!

### 4. Tokenization
Setup some tokenization settings like left padding because it makes [training use less memory](https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa):

In [6]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

Setup the tokenize function to make labels and input_ids the same. This is basically what [self-supervised fine-tuning](https://neptune.ai/blog/self-supervised-learning) is:

In [7]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=700,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

And run convert each data_point into a prompt that I found online that works quite well:

In [8]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = data_point["full_prompt"]
    return tokenize(full_prompt)

Reformat to prompt and tokenize each sample:

In [9]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Loading cached processed dataset at /home/sam/.cache/huggingface/datasets/json/default-8e74ab5de84b7560/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-febea880da879e7b.arrow


Map:   0%|          | 0/568 [00:00<?, ? examples/s]

### 5. Setup Lora

In [10]:
model.train() # put model back into training mode
model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:17                                                                                   │
│                                                                                                  │
│   14 │   bias="none",                                                                            │
│   15 │   task_type="CAUSAL_LM",                                                                  │
│   16 )                                                                                           │
│ ❱ 17 model = get_peft_model(model, config)                                                       │
│   18                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/peft/mapping.py:147 in get_peft_model                    │
│                                                                                                  │
│   144 │   │   peft_config = _prepare_lora_config(peft_config, model_config)                      │
│   145 │   else:                                                                                  │
│   146 │   │   peft_config = _prepare_prompt_learning_config(peft_config, model_config)           │
│ ❱ 147 │   return MODEL_TYPE_TO_PEFT_MODEL_MAPPING[peft_config.task_type](model, peft_config)     │
│   148                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/peft/peft_model.py:560 in __init__                       │
│                                                                                                  │
│    557 │   """                                                                                   │
│    558 │                                                                                         │
│    559 │   def __init__(self, model, peft_config: PeftConfig):                                   │
│ ❱  560 │   │   super().__init__(model, peft_config)                                              │
│    561 │   │   self.base_model_prepare_inputs_for_generation = self.base_model.prepare_inputs_f  │
│    562 │                                                                                         │
│    563 │   def forward(                                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/peft/peft_model.py:79 in __init__                        │
│                                                                                                  │
│     76 │   │   if isinstance(self.peft_config, PromptLearningConfig):                            │
│     77 │   │   │   self._setup_prompt_encoder()                                                  │
│     78 │   │   else:                                                                             │
│ ❱   79 │   │   │   self.base_model = LoraModel(peft_config, model)                               │
│     80 │   │   if getattr(self.peft_config, "modules_to_save", None) is not None:                │
│     81 │   │   │   self.modules_to_save = self.peft_config.modules_to_save                       │
│     82 │   │   │   _set_trainable(self)                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:133 in __init__                      │
│                                                                                                  │
│   130 │   │   super().__init__()                           

Optional stuff to setup Weights and Biases to view training graphs:

In [11]:
wandb_project = "testing-dataloaders"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project


In [12]:
if torch.cuda.device_count() > 1:
    # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
    model.is_parallelizable = True
    model.model_parallel = True

In [13]:
from transformers import EvalPrediction
import torch.nn as nn

def compute_metrics(eval_pred):
    print("COMPUTING METRICS")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### 6. Training arguments
If you run out of GPU memory, change per_device_train_batch_size. The gradient_accumulation_steps variable should ensure this doesn't affect batch dynamics during the training run. All the other variables are standard stuff that I wouldn't recommend messing with:

In [14]:
batch_size = 128
per_device_train_batch_size = 1
gradient_accumulation_steps = batch_size // per_device_train_batch_size
output_dir = "testing-dataloader"

training_args = TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
        max_steps=400,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=1,
        optim="adamw_torch",
        evaluation_strategy="steps", # if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=1,
        save_steps=50,
        output_dir=output_dir,
        # save_total_limit=3,
        load_best_model_at_end=False,
        # ddp_find_unused_parameters=False if ddp else None,
        group_by_length=True, # group sequences of roughly the same length together to speed up training
        report_to="none", # if use_wandb else "none",
        run_name=f"codellama-{datetime.now().strftime('%Y-%m-%d-%H-%M')}", # if use_wandb else None,
    )

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
    compute_metrics=compute_metrics
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:28                                                                                   │
│                                                                                                  │
│   25 │   │   run_name=f"codellama-{datetime.now().strftime('%Y-%m-%d-%H-%M')}", # if use_wand    │
│   26 │   )                                                                                       │
│   27                                                                                             │
│ ❱ 28 trainer = Trainer(                                                                          │
│   29 │   model=model,                                                                            │
│   30 │   train_dataset=tokenized_train_dataset,                                                  │
│   31 │   eval_dataset=tokenized_val_dataset,                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:412 in __init__                  │
│                                                                                                  │
│    409 │   │                                                                                     │
│    410 │   │   # At this stage the model is already loaded                                       │
│    411 │   │   if _is_quantized_and_base_model and not _is_peft_model:                           │
│ ❱  412 │   │   │   raise ValueError(                                                             │
│    413 │   │   │   │   "You cannot perform fine-tuning on purely quantized models. Please attac  │
│    414 │   │   │   │   " the quantized model to correctly perform fine-tuning. Please see: http  │
│    415 │   │   │   │   " for more details"                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of 
the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for
more details

In [15]:
print(model.forward.__doc__)


   The [`GPT2LMHeadModel`] forward method, overrides the `__call__` special method.

    <Tip>

    Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
    instance afterwards instead of this since the former takes care of running the pre and post processing steps while
    the latter silently ignores them.

    </Tip>

    Args:
        input_ids (`torch.LongTensor` of shape `(batch_size, input_ids_length)`):
            `input_ids_length` = `sequence_length` if `past_key_values` is `None` else
            `past_key_values[0][0].shape[-2]` (`sequence_length` of input past key value states). Indices of input
            sequence tokens in the vocabulary.

            If `past_key_values` is used, only `input_ids` that do not have their past calculated should be passed as
            `input_ids`.

            Indices can be obtained using [`AutoTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.

Then we do some pytorch-related optimisation (which just make training faster but don't affect accuracy):

In [15]:
model.config.use_cache = False

old_state_dict = model.state_dict
model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
    model, type(model)
)
if torch.__version__ >= "2" and sys.platform != "win32":
    print("compiling the model")
    model = torch.compile(model)

compiling the model


In [16]:
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'trainer' is not defined

In [16]:
import sys
print(sys.executable)

/opt/conda/bin/python
